<a href="https://colab.research.google.com/github/Yogesh914/emogo-ml/blob/main/gemma_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluating In-Context Learning With Gemma-7b 🦋

## Intial setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers pydub accelerate bitsandbytes hf_transfer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 67.7 MB/s eta 0:00:00


In [3]:
!pip install -U transformers

In [4]:
!pip install noisereduce

In [5]:
import torch
import transformers
from IPython.display import Markdown, display
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import userdata
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from moviepy.editor import VideoFileClip
import numpy as np
import os
from pydub import AudioSegment
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import noisereduce as nr

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Transcription Using Whisper v3 (large)

In [7]:
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, use_safetensors=True, torch_dtype=torch.float16, low_cpu_mem_usage=True
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=440,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    device=device,
    torch_dtype=torch.float16
)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
def process_video(video_file):
    if video_file.endswith('.mp4'):
        audio_segment = AudioSegment.from_file(video_file, format="mp4")
    else:
        audio_segment = AudioSegment.from_file(video_file, format="mov")
    audio_segment = audio_segment.set_frame_rate(16000)
    audio_array = np.array(audio_segment.get_array_of_samples())

    if audio_segment.channels == 2:
        audio_array = audio_array.reshape((-1, 2))
        audio_array = audio_array.mean(axis=1)
    audio_array = audio_array.astype(np.float32) / (2**15)

    audio_array = nr.reduce_noise(y=audio_array, sr=16000)

    return audio_array, audio_segment.frame_rate

In [9]:
class AudioDataset(Dataset):
    def __init__(self, data_folder):
        self.data_folder = data_folder
        self.video_files = sorted([os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('.mp4') or f.endswith('.MOV')])

    def __len__(self):
        return len(self.video_files)

    def __getitem__(self, idx):
        video_file = self.video_files[idx]
        audio_array, sample_rate = process_video(video_file)
        audio_tensor = torch.from_numpy(audio_array)
        return audio_tensor, sample_rate

In [10]:
import warnings

warnings.filterwarnings("ignore")

In [11]:
data_folder = '/content/drive/MyDrive/data/beta_vids'
dataset = AudioDataset(data_folder)

batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

transcriptions = []
for batch in dataloader:
    audio_tensors, sample_rates = batch
    audio_tensor = audio_tensors[0]
    sample_rate = sample_rates[0].item()
    audio_numpy = audio_tensor.numpy()
    audio_dict = {"path": "audio_file", "array": audio_numpy, "sampling_rate": sample_rate}
    result = pipe(audio_dict, batch_size=1)
    transcription = result["text"]
    transcriptions.append(transcription)

df_transcriptions = pd.DataFrame({'Transcriptions': transcriptions})
df_transcriptions

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


,Transcriptions
0,I just woke up and went to Four Lake Dining H...
1,It's 12 o'clock and I'm going to the dining h...
2,My class is beginning in 5 minutes and I just...
3,I just had lunch and my class will begin in f...
4,I'm right now at the dining hall and I just f...
...,...
58,I'm feeling okay because I is the meat of the...
59,I am feeling okay.
60,I feel pretty good. I have a little bit icky ...
61,I'm feeling not okay. I just have a lot to......


In [34]:
df_transcriptions.to_csv('/content/drive/MyDrive/data/transcriptions.csv')

## Curating Dataset

In [ ]:
ema_survey = pd.read_csv(os.path.join("/content/drive/MyDrive/data/emogo_full_study.csv"))
filtered_columns = ['ema_aware', 'ema_support', 'ema_insight', 'ema_fulfilled', 'ema_hopeless', 'ema_anxious', 'Trigger.Index']
df_filtered = ema_survey[filtered_columns]

df_final = df_filtered[ema_survey['User.Email'] == '']
df_final = df_final.dropna()
df_final.reset_index(drop=True, inplace=True)

In [ ]:
merge = pd.concat([df_captions, df_final], axis=1)
merge.to_csv(os.path.join("/content/drive/MyDrive/Colab Notebooks/data/merged_captions.csv"), index=False)
merge

## Creating Custom Prompt

In [ ]:
df = merge

def format_prompt(row):
    return f"Caption: {row['Captions']}\n" \
           f"Had you noticed you were feeling this way before we asked?: {int(row['ema_aware'])}\n" \
           f"Did you feel you were supported by others?: {int(row['ema_support'])}\n" \
           f"Did you recognize how your feelings were influencing your outlook on things?: {int(row['ema_insight'])}\n" \
           f"How fulfilled did you feel?: {int(row['ema_fulfilled'])}\n" \
           f"How hopeless did you feel?: {int(row['ema_hopeless'])}\n" \
           f"How anxious did you feel?: {int(row['ema_anxious'])}"

prompt = "\n\n".join(df.iloc[3:6].apply(format_prompt, axis=1))

instructions = "\n\nBased on the previous entries, predict the ratings for the following caption on a scale of 1 to 5 and make sure to give only the your answer in json format and nothing else:"
last_caption = df.iloc[-1]['Captions']

prompt += f"{instructions}\n\nCaption: {last_caption}\n" \
          "Had you noticed you were feeling this way before we asked?: \n" \
          "Did you feel you were supported by others?: \n" \
          "Did you recognize how your feelings were influencing your outlook on things?: \n" \
          "How fulfilled did you feel?: \n" \
          "How hopeless did you feel?: \n" \
          "How anxious did you feel?: "

print(prompt)

## Prompting Gemma-7b-it

In [ ]:
os.environ ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'
torch.set_default_device(device)

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it", token=userdata.get('hgemma'))

model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it",
                                             torch_dtype="auto",
                                             device_map="auto", token=userdata.get('hgemma')
                                             )

In [ ]:
chat = [
    { "role": "user", "content": prompt },
]

input = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
input

In [ ]:
inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")

outputs = model.generate(input_ids=inputs.to("cuda"),
                         max_new_tokens=512)

text = tokenizer.decode(outputs[0],skip_special_tokens=True, clean_up_tokenization_spaces=True)
display(Markdown(text))